In [ ]:
%load_ext autoreload
%autoreload 2

# Derived Variables Demo: Heating & Cooling Degree Days

This notebook demonstrates how to use derived variables in ClimakitAE, focusing on **heating degree days (HDD)** and **cooling degree days (CDD)** for energy demand analysis in Los Angeles.

## What are Degree Days?

- **Heating Degree Days (HDD)**: Measure of energy demand for heating when outdoor temperature falls below 65°F
- **Cooling Degree Days (CDD)**: Measure of energy demand for cooling when outdoor temperature rises above 65°F

These metrics are essential for:
- Building energy modeling
- Utility demand forecasting
- Climate adaptation planning
- Energy efficiency assessments

In [10]:
import numpy as np
from dask.diagnostics import ProgressBar

import climakitae as ck

# Initialize ClimateData
cd = ck.ClimateData(verbosity=-1)

## Why Derived Variables? Old vs New Approach

### ❌ Previous Approach (Legacy Interface)

```mermaid
flowchart TD
    A[Step 1: Fetch first variable<br/>tasmax_data = get_data] --> B[Step 2: Fetch second variable<br/>tasmin_data = get_data]
    B --> C[Step 3: Define calculation function<br/>def calc_cdd]
    C --> D[Step 4: Manually compute<br/>cdd = calc_cdd]
    D --> E[Step 5: Manual post-processing<br/>cdd_clipped = cdd.sel<br/>cdd_annual = cdd.resample]
    E --> F[⏱️ RESULT]
    
    style A fill:#ffcccc,stroke:#cc0000,stroke-width:2px
    style B fill:#ffcccc,stroke:#cc0000,stroke-width:2px
    style C fill:#ffcccc,stroke:#cc0000,stroke-width:2px
    style D fill:#ffcccc,stroke:#cc0000,stroke-width:2px
    style E fill:#ffcccc,stroke:#cc0000,stroke-width:2px
    style F fill:#ffeeee,stroke:#cc0000,stroke-width:2px
```

**Pain Points:**
- 💥 Multiple `get_data()` calls might load full datasets into memory (lazy dataset handling was inconsistent)
- 🔧 Manual coordination of multiple variables
- 📦 Post-processing happens after data is loaded
- 🔁 No reusability - repeat for each analysis
- ⚠️ Error-prone parameter matching

---

### ✅ New Approach (Derived Variables System)

```mermaid
flowchart TD
    A[Step 1: Define function ONCE and register<br/>@register_derived] --> B[Step 2: Single call with processing<br/>ClimateData.variable.processes.get]
    B --> C[⚡ LAZY RESULT<br/>computed on demand]
    
    style A fill:#ccffcc,stroke:#00cc00,stroke-width:2px
    style B fill:#ccffcc,stroke:#00cc00,stroke-width:2px
    style C fill:#eeffee,stroke:#00cc00,stroke-width:3px
```

**Benefits:**
- ✨ **Single `.get()` call** - All processing in one query
- 🔄 **Automatic variable fetching** - System knows CDD needs t2max & t2min
- 💾 **Lazy evaluation** - Data not loaded until needed
- 🎯 **Processing pipeline** - Clipping, time slicing happen efficiently
- ♻️ **Reusable** - Define once, use everywhere
- 🛡️ **Type-safe** - Registry validates dependencies
- 📦 **Composable** - Chain with other processors seamlessly

---

### Code Comparison

**Old Approach:**
```python
# Step 1 & 2: Multiple data fetches
tasmax_data = get_data({'variable': 'tasmax', 'area': 'Los Angeles', ...})
tasmin_data = get_data({'variable': 'tasmin', 'area': 'Los Angeles', ...})

# Step 3 & 4: Manual calculation
def calc_cdd(tasmax, tasmin):
    t_avg = (tasmax + tasmin) / 2
    return np.maximum(0, t_avg - 291.48)  # 65°F threshold

cdd = calc_cdd(tasmax_data, tasmin_data)

# Step 5: Manual post-processing
cdd_clipped = cdd.sel(lat=..., lon=...)
cdd_annual = cdd_clipped.resample(time='Y').sum()
```

**New Approach:**
```python
# Step 1: Register once (already done in builtin!)
@register_derived(variable='CDD_wrf', query={'variable_id': ['t2max', 't2min']})
def calc_cdd_wrf(ds): ...

# Step 2: Single call - that's it!
cdd_data = (
    ClimateData()
    .variable('CDD_wrf')           # Derived variable!
    .activity_id('WRF')
    .processes({
        'clip': 'Los Angeles County',
        'time_slice': (2020, 2050)
    })
    .get()
)
# Returns lazy dataset with all processing applied
```

In [4]:
# Show all available derived variables
# Notice the HDD and CDD variables for both WRF and LOCA2 data
cd.show_derived_variables()

2026-01-22 07:51:00 - climakitae.new_core.user_interface - INFO - Derived Variables (computed from source variables during loading):
2026-01-22 07:51:00 - climakitae.new_core.user_interface - INFO - ------------------------------------------------------------------
Derived Variables (computed from source variables during loading):
------------------------------------------------------------------
2026-01-22 07:51:00 - climakitae.new_core.user_interface - INFO - CDD_loca                        depends on: tasmax, tasmin 
                                  └─ Cooling degree days from LOCA2 data (base 65°F)
CDD_loca                        depends on: tasmax, tasmin 
                                  └─ Cooling degree days from LOCA2 data (base 65°F)
2026-01-22 07:51:00 - climakitae.new_core.user_interface - INFO - CDD_wrf                         depends on: t2 
                                  └─ Cooling degree days from WRF data (base 65°F)
CDD_wrf                         depends on: t2 

## 2. Understanding the Degree Days Variables

The degree days metrics are built-in derived variables that automatically fetch and compute from temperature data:

In [5]:
from climakitae.new_core.derived_variables import list_derived_variables

# Get info about the degree days variables
derived_vars = list_derived_variables()

degree_days_vars = ['HDD_wrf', 'CDD_wrf', 'HDD_loca', 'CDD_loca']

for var_name in degree_days_vars:
    if var_name in derived_vars:
        info = derived_vars[var_name]
        print(f"\n{var_name}:")
        print(f"  Depends on: {info.depends_on}")
        print(f"  Description: {info.description}")


HDD_wrf:
  Depends on: ['t2']
  Description: Heating degree days from WRF data (base 65°F)

CDD_wrf:
  Depends on: ['t2']
  Description: Cooling degree days from WRF data (base 65°F)

HDD_loca:
  Depends on: ['tasmax', 'tasmin']
  Description: Heating degree days from LOCA2 data (base 65°F)

CDD_loca:
  Depends on: ['tasmax', 'tasmin']
  Description: Cooling degree days from LOCA2 data (base 65°F)


## 3. Fetching Cooling Degree Days for Los Angeles

Let's fetch CDD data for Los Angeles County to analyze cooling energy demand under future warming:

In [ ]:
cd = ck.ClimateData(verbosity=-2)

# Fetch cooling degree days for Los Angeles County
cdd_data = (
    cd
    .catalog("cadcat")
    .variable("CDD_wrf")  # Cooling degree days from WRF
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")  # 3km resolution
    # .experiment_id("ssp370")  # High emissions scenario
    .processes({
        "warming_level": {
            "warming_levels": [1.2, 2.0, 3.0]
        },
        "clip": "Los Angeles County",
    })
    .get()
)

/home/nschroed/Work/climakitae/climakitae/util/utils.py:1981: UserWarning: 

Incomplete warming level for WRF.UCLA.EC-Earth3-Veg.ssp370.day.d03.r1i1p1f1 at 1.2C. 
Skipping this warming level.
  warnings.warn(


In [11]:
# Add dummy time dimension to warming level data for easier handling
def add_dummy_time_to_wl(data):
    """
    Convert time_delta dimension to a dummy time dimension for easier handling.
    """
    import xarray as xr
    import pandas as pd

    if 'time_delta' in data.dims:
        wl_values = data['time_delta'].values
        time_values = pd.to_datetime(wl_values, unit='D', origin='2025-01-01')
        data = data.rename({'time_delta': 'time'})
        data = data.assign_coords(time=time_values)
    return data

cdd_data = add_dummy_time_to_wl(cdd_data)
cdd_data

<xarray.Dataset> Size: 6GB
Dimensions:            (warming_level: 3, x: 64, y: 72, sim: 5, time: 10950)
Coordinates:
  * warming_level      (warming_level) float64 24B 1.2 2.0 3.0
  * x                  (x) float64 512B -4.215e+06 -4.212e+06 ... -4.026e+06
  * y                  (y) float64 576B 7.369e+05 7.399e+05 ... 9.499e+05
  * sim                (sim) object 40B 'WRF_UCLA_EC-Earth3-Veg_ssp370_day_d0...
  * time               (time) datetime64[ns] 88kB 2010-01-05 ... 2039-12-28
    lakemask           (y, x) float32 18kB dask.array<chunksize=(72, 2), meta=np.ndarray>
    landmask           (y, x) float32 18kB dask.array<chunksize=(72, 2), meta=np.ndarray>
    lat                (y, x) float32 18kB dask.array<chunksize=(72, 2), meta=np.ndarray>
    lon                (y, x) float32 18kB dask.array<chunksize=(72, 2), meta=np.ndarray>
    centered_year      (sim, warming_level) float64 120B 2.025e+03 ... 2.058e+03
    Lambert_Conformal  int64 8B 0
Data variables:
    CDD_wrf            (sim, warming_level, time, y, x) float64 6GB dask.array<chunksize=(1, 1, 2433, 44, 2), meta=np.ndarray>
Attributes: (12/119)
    AERCU_FCT:                        1.0
    AERCU_OPT:                        0
    AUTO_LEVELS_OPT:                  2
    BL_PBL_PHYSICS:                   1
    BOTTOM_TOP_GRID_DIMENSION:        40
    BOTTOM_TOP_PATCH_END_STAG:        40
    ...                               ...
    warming_level:                    {'warming_levels': [1.2, 2.0, 3.0]}
    clip:                             Process 'clip' applied to the data. Cli...
    update_attributes:                Process 'update_attributes' applied to ...
    filter_unadjusted_models:         yes
    concat:                           Process 'concat' applied to the data. T...
    warming_level_simple:             Process 'warming_level_simple' applied ...

## 4. Visualizing Cooling Degree Days: Spatial Based

Let's calculate and visualize annual cooling degree days:

In [ ]:
# Count days with cooling degree days > 0 per year
cooling_days_per_year = cdd_data["CDD_wrf"].resample(time="Y").sum()

# Average years for spatial plot
spt = cooling_days_per_year.mean(dim="time") 

# Compute with progress bar
with ProgressBar():
    spt = spt.median(dim='sim').compute()

In [ ]:
import matplotlib.pyplot as plt

# Create figure with 3 subplots (one per warming level)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

warming_levels = spt.warming_level.values

for idx, (ax, wl) in enumerate(zip(axes, warming_levels)):
    # Select data for this warming level
    wl_data = spt.sel(warming_level=wl)
    
    # Create contourf plot
    im = wl_data.plot.contourf(
        ax=ax, x='lon', y='lat',
        cmap='YlOrRd', levels=100,
        add_colorbar=False,
        vmin=1,
        vmax=300
    )
    
    ax.set_title(f'{wl}°C Warming Level', fontsize=13, fontweight='bold')
    ax.set_xlabel('Longitude', fontsize=11)
    ax.set_ylabel('Latitude', fontsize=11)

# Add shared colorbar
fig.colorbar(im, ax=axes, label='Annual Cooling Days', pad=0.02, shrink=0.8)

fig.suptitle('Cooling Days (65 °F) Distribution Across Warming Levels\nLos Angeles County', 
             fontsize=15, fontweight='bold', y=1.02)

## 5. Temporal Trends by Warming Level

Let's visualize how cooling days evolve over time for each warming level:

In [ ]:
# Average over spatial dimensions for county-wide average
from dask.diagnostics import ProgressBar
with ProgressBar():
    cooling_days_per_year = cooling_days_per_year.mean(dim=["y", "x"]).compute()

# Plot time series for each warming level
fig, ax = plt.subplots(figsize=(14, 7))

# Define colors for each warming level
colors = ['#2E86AB', '#A23B72', '#F18F01']  # Blue, Purple, Orange
warming_levels = cooling_days_per_year.warming_level.values

# Plot ensemble mean for each warming level
for idx, wl in enumerate(warming_levels):
    wl_data = cooling_days_per_year.sel(warming_level=wl)
    
    # Plot individual simulations with low alpha
    for sim in range(wl_data.sizes['sim']):
        sim_data = wl_data.isel(sim=sim)
        ax.plot(sim_data.time.dt.year, sim_data.values, 
                alpha=0.15, color=colors[idx], linewidth=0.8)
    
    # Plot ensemble mean with solid line
    ensemble_mean = wl_data.mean(dim='sim')
    ax.plot(ensemble_mean.time.dt.year, ensemble_mean.values,
            color=colors[idx], linewidth=3, label=f'{wl}°C Warming', 
            marker='o', markersize=4, markevery=5)

ax.set_xlabel('Year', fontsize=13, fontweight='bold')
ax.set_ylabel('Annual Cooling Days', fontsize=13, fontweight='bold')
ax.set_title('Annual Cooling Days by Global Warming Level\nLos Angeles County (UCLA WRF)', 
             fontsize=15, fontweight='bold', pad=15)
ax.legend(loc='upper left', fontsize=11, framealpha=0.95)
ax.grid(True, alpha=0.3, linestyle='--')

# Add annotation
ax.text(0.98, 0.02, 'Higher warming → More cooling days', 
        transform=ax.transAxes, fontsize=10, 
        verticalalignment='bottom', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.tight_layout()

In [ ]:
# Summary statistics for cooling days by warming level
for wl in warming_levels:
    wl_data = cooling_days_per_year.sel(warming_level=wl).mean(dim='sim')
    mean_days = wl_data.mean().values
    print(f"\n{wl}°C Warming Level:")
    print(f"  Average cooling days per year: {mean_days:.0f} days")
    print(f"  Range: {wl_data.min().values:.0f} - {wl_data.max().values:.0f} days")

## 6. Customizing CDD

There are two ways to customize the calculation of Cooling Degree Days:
1. Use the built-in methods to pass your own threshold and register it
2. Define your own Cooling Degree Days method and register it

### 6.1 Customizing CDD: Using Built-in Functions with Custom Thresholds

The built-in `calc_cdd_wrf` and `calc_hdd_wrf` functions accept optional threshold arguments:
- `threshold_f` - threshold in Fahrenheit
- `threshold_c` - threshold in Celsius  
- `threshold_k` - threshold in Kelvin

**Best Practice:** Wrap the builtin function with your threshold and register it as a new variable. This:
- ✅ Creates a distinct, named variable (`CDD_wrf_75f`)
- ✅ Is reusable across sessions (if added to your project)
- ✅ Doesn't mutate global state
- ✅ Keeps the variable name meaningful

In [ ]:
from climakitae.new_core.derived_variables.registry import register_user_function
from climakitae.new_core.derived_variables.builtin.temperature import calc_cdd_wrf

# The builtin calc_cdd_wrf accepts threshold_f as an argument
# We can wrap it with a custom threshold and register as a new variable

def calc_cdd_wrf_75f(ds):
    """CDD with 75°F threshold - wraps builtin with custom threshold."""
    return calc_cdd_wrf(ds, threshold_f=75.0)

register_user_function(
    name="CDD_wrf_75f",
    depends_on=["t2"],  # Same dependency as CDD_wrf
    func=calc_cdd_wrf_75f,
    description="Cooling Degree Days from WRF with 75°F base",
    units="K",
)

In [ ]:
cd = ck.ClimateData(verbosity=-2)

# Fetch cooling degree days with 75°F threshold
cdd_data_75f = (
    cd
    .catalog("cadcat")
    .variable("CDD_wrf_75f")  # Now using our custom 75°F variable
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")
    .processes({
        "warming_level": {
            "warming_levels": [1.2, 2.0, 3.0]
        },
        "clip": "Los Angeles County",
    })
    .get()
)

cdd_data_75f = add_dummy_time_to_wl(cdd_data_75f)

In [ ]:
# Count days with cooling degree days > 0 per year
cooling_days_per_year = cdd_data_75f["CDD_wrf_75f"].resample(time="Y").sum()

# Average years for spatial plot
spt = cooling_days_per_year.mean(dim="time") 

# Compute with progress bar
with ProgressBar():
    spt = spt.median(dim='sim').compute()

In [ ]:
# Create figure with 3 subplots (one per warming level)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

warming_levels = spt.warming_level.values

for idx, (ax, wl) in enumerate(zip(axes, warming_levels)):
    # Select data for this warming level
    wl_data = spt.sel(warming_level=wl)
    
    # Create contourf plot
    im = wl_data.plot.contourf(
        ax=ax, x='lon', y='lat',
        cmap='YlOrRd', levels=100,
        add_colorbar=False,
        vmin=1,
        vmax=300
    )
    
    ax.set_title(f'{wl}°C Warming Level', fontsize=13, fontweight='bold')
    ax.set_xlabel('Longitude', fontsize=11)
    ax.set_ylabel('Latitude', fontsize=11)

# Add shared colorbar
fig.colorbar(im, ax=axes, label='Annual Cooling Days', pad=0.02, shrink=0.8)

fig.suptitle('Cooling Days (75 °F) Distribution Across Warming Levels\nLos Angeles County', 
             fontsize=15, fontweight='bold', y=1.02)

#### Factory Pattern: Creating Multiple Threshold Variables

For projects needing multiple thresholds, use a factory function:

In [ ]:
# Factory function to create CDD/HDD variables with arbitrary thresholds
def create_degree_day_variable(
    name: str,
    threshold_f: float,
    metric: str = "CDD",  # "CDD" or "HDD"
    data_source: str = "wrf",  # "wrf" or "loca"
):
    """
    Factory to create degree day variables with custom thresholds.
    
    Parameters
    ----------
    name : str
        Name for the new derived variable (e.g., "CDD_wrf_80f")
    threshold_f : float
        Threshold temperature in Fahrenheit
    metric : str
        "CDD" for cooling degree days, "HDD" for heating degree days
    data_source : str
        "wrf" for WRF data (depends on t2), "loca" for LOCA2 (depends on tasmax, tasmin)
    """
    from climakitae.new_core.derived_variables.builtin.temperature import (
        calc_cdd_wrf, calc_hdd_wrf, calc_cdd_loca, calc_hdd_loca
    )
    
    # Select the appropriate function and dependencies
    func_map = {
        ("CDD", "wrf"): (calc_cdd_wrf, ["t2"]),
        ("HDD", "wrf"): (calc_hdd_wrf, ["t2"]),
        ("CDD", "loca"): (calc_cdd_loca, ["tasmax", "tasmin"]),
        ("HDD", "loca"): (calc_hdd_loca, ["tasmax", "tasmin"]),
    }
    
    base_func, depends_on = func_map[(metric, data_source)]
    
    def custom_func(ds):
        return base_func(ds, threshold_f=threshold_f)
    
    register_user_function(
        name=name,
        depends_on=depends_on,
        func=custom_func,
        description=f"{metric} ({data_source.upper()}) with {threshold_f}°F base",
        units="K",
    )
    print(f"✓ Registered '{name}' ({metric} at {threshold_f}°F)")

# Example: Create variables for multiple thresholds
create_degree_day_variable("CDD_wrf_70f", threshold_f=70.0, metric="CDD", data_source="wrf")
create_degree_day_variable("CDD_wrf_80f", threshold_f=80.0, metric="CDD", data_source="wrf")
create_degree_day_variable("HDD_wrf_60f", threshold_f=60.0, metric="HDD", data_source="wrf")

### 6.2: Define your own cooling degree days function

In [ ]:
from climakitae.new_core.derived_variables.registry import register_user_function
from climakitae.new_core.derived_variables.utils import f_to_k

THRESHOLD_F = 75.0
threshold_k = f_to_k(THRESHOLD_F)

def calc_custom_cdd(ds):
    """
    A custom CDD function that runs on a different variable. 

    In this example we look at days where the daily minimum temperature exceeds 75°F.
    
    IMPORTANT: The output variable name must match the registered name!
    """
    import xarray as xr
    
    # Calculate degree days above threshold
    cdd_values = np.maximum(0, ds.t2min - threshold_k)
    
    # Replace zeros with NaN (days below threshold become NaN)
    cdd_values = xr.where(cdd_values > 0, cdd_values, np.nan)
    
    ds["CDD_min_wrf_75f"] = cdd_values
    ds["CDD_min_wrf_75f"].attrs = {
        "units": "K",
        "long_name": "Cooling Degree Days (WRF) - Min Temp > 75°F",
        "comment": f"Cooling degree days calculated from daily minimum temperature with base {threshold_k} K. Days below threshold are NaN.",
        "derived_from": "t2min",
        "derived_by": "climakitae",
        "threshold": f"{threshold_k} K",
    }
    return ds

register_user_function(
    name="CDD_min_wrf_75f",
    depends_on=["t2min"],
    func=calc_custom_cdd,
    description="Cooling Degree Days from WRF with 75°F base using daily minimum temperature",
    units="K",
)

In [ ]:
cd = ck.ClimateData(verbosity=-2)

# Fetch cooling degree days for Los Angeles County
cdd_data_75f = (
    cd
    .catalog("cadcat")
    .variable("CDD_min_wrf_75f")  # Cooling degree days from WRF with 75°F base
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id("day")
    .grid_label("d03")  # 3km resolution
    # .experiment_id("ssp370")  # High emissions scenario
    .processes({
        "warming_level": {
            "warming_levels": [1.2, 2.0, 3.0]
        },
        "clip": "Los Angeles County",
    })
    .get()
)

cdd_data_75f = add_dummy_time_to_wl(cdd_data_75f)

In [ ]:
# Count days with cooling degree days > 0 per year
cooling_days_per_year = cdd_data_75f["CDD_min_wrf_75f"].resample(time="Y").sum()

# Average years for spatial plot
spt = cooling_days_per_year.mean(dim="time") 

# Compute with progress bar
with ProgressBar():
    spt = spt.median(dim='sim').compute()

In [ ]:
# Create figure with 3 subplots (one per warming level)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

warming_levels = spt.warming_level.values

for idx, (ax, wl) in enumerate(zip(axes, warming_levels)):
    # Select data for this warming level
    wl_data = spt.sel(warming_level=wl)
    
    # Create contourf plot
    im = wl_data.plot.contourf(
        ax=ax, x='lon', y='lat',
        cmap='YlOrRd', levels=100,
        add_colorbar=False,
        vmin=1,
        vmax=300
    )
    
    ax.set_title(f'{wl}°C Warming Level', fontsize=13, fontweight='bold')
    ax.set_xlabel('Longitude', fontsize=11)
    ax.set_ylabel('Latitude', fontsize=11)

# Add shared colorbar
fig.colorbar(im, ax=axes, label='Annual Cooling Days', pad=0.02, shrink=0.8)

fig.suptitle('Cooling Days (75 °F) Distribution Across Warming Levels\nLos Angeles County', 
             fontsize=15, fontweight='bold', y=1.02)

## 7. Summary: Key Takeaways

**What We've Learned:**

1. **Derived variables simplify analysis** - Single `.get()` call automatically fetches and computes HDD/CDD from temperature data

2. **Warming levels provide climate-relevant framing** - Analyzing by 1.2°C, 2.0°C, and 3.0°C warming shows progressive impacts

3. **Los Angeles shows clear cooling demand increase** - CDD rises significantly with warming while HDD decreases

4. **Spatial patterns matter** - Inland areas experience more extreme cooling demands than coastal regions

5. **Energy costs will increase** - Higher warming levels translate directly to higher HVAC energy costs

6. **Customizing Functions** - Metrics can be customized and added to the catalog (locally, this does not change the catalog other users see). 

## 8. Available Degree Days Variables

| Variable | Data Source | Input Variables | Description |
|----------|-------------|-----------------|-------------|
| `HDD_wrf` | WRF | t2 | Heating degree days (base 65°F) |
| `CDD_wrf` | WRF | t2 | Cooling degree days (base 65°F) |
| `HDD_loca` | LOCA2 | tasmax, tasmin | Heating degree days (base 65°F) |
| `CDD_loca` | LOCA2 | tasmax, tasmin | Cooling degree days (base 65°F) |

All variables automatically:
- Fetch required temperature data
- Calculate daily average: (max + min) / 2
- Apply 65°F (291.48K) threshold
- Return lazy-evaluated xarray datasets